<a href="https://colab.research.google.com/github/korkeatw/thai-healthcare-fakenews-detection/blob/main/thai_healthcare_fakenews_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyICU==2.4.3
!pip install pythainlp

     |████████████████████████████████| 219 kB 5.1 MB/s 
  Created wheel for PyICU: filename=PyICU-2.4.3-cp37-cp37m-linux_x86_64.whl size=1260863 sha256=97c761702d0399ada4cc1f5ac93ebe035fe6186c7ee0fbe52cfb41314409da9e
  Stored in directory: /root/.cache/pip/wheels/65/b3/68/b4ea8f9ec4ef88063a1d70886fcb62f17ea7dbb2c09e4da37b
Successfully built PyICU
     |████████████████████████████████| 11.5 MB 6.5 MB/s 


In [ ]:
import pandas as pd
import sklearn

In [ ]:
# Paper: https://ieeexplore.ieee.org/document/9678187/
# Dataset: https://github.com/byinth/LimeSoda

df=pd.read_json('https://raw.githubusercontent.com/byinth/LimeSoda/main/LimeSoda/Limesoda.jsonl', lines=True)
df

,Title,Detail,Title Token Tags,Detail Token Tags,Document Tag
0,None,"[ประเทศจีน, ชอบ, ปลอม, เอา, ยาง, รถ, ไป, ทำ, ไ...",None,"[I-Influencer, Fb-Refer, Fb-Refer, Fb-Refer, F...",Fake News
1,None,"[ประกาศ, กฎอัยการศึก, ปิด, การ, เข้า, ออก, ทุก...",None,"[T-Clickbait, T-Clickbait, , , , , , , , , , ,...",Undefined
2,"[ประโยชน์, มาก, ก็, ใช่, ว่า, จะ, ไม่มี, โทษ, ...","[แม้ว่า, ทุเรียน, จะ, มี, รสชาติ, ที่, ถูก, อก...","[, , , , , , , , , , M-Convincing, M-Convincin...","[, , , , , , , , , , , , , , , , , , , , , , ,...",Undefined
3,None,"[มั่นใจ, กระซิบหูหนู, พระพิฆเนศ, ไม่ติด, เชื้อ...",None,"[T-Clickbait, T-Clickbait, T-Clickbait, T-Clic...",Fake News
4,None,"[พวก, ที่, เชื่อ, ว่า, คน, ไม่, ป่วย, ไม่, ต้อ...",None,"[, , , , , , , , , , , , , , , , , , , , , , ,...",Undefined
...,...,...,...,...,...
7134,None,"[ข่าว, ปลอม, , , อย่า, แชร์, !, , , ❌, , , เผย...",None,None,Fact News
7135,"[ข่าว, ปลอม, อย่า, แชร์, !, ปรากฏการณ์, Equino...","[ตาม, ที่, มี, ข่าว, ปรากฏ, ใน, สื่อ, ต่างๆ, เ...",None,None,Fact News
7136,None,"[บริโภค, , , “, เครื่อง, ดื่ม, ”, , , อย่าง, ไ...",None,None,Fact News
7137,"[แพทย์, เตือน, แคะ, หู, บ่อย, อันตราย, !, เสี่...","[นพ.สมศักดิ์อรรฆศิลป์, อธิบดี, กรมการแพทย์, กร...",None,None,Fact News


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Detail'], df['Document Tag'], test_size=.2, random_state=0, shuffle=True)


# Find the Best Algorithm

หา algorithm ที่ได้ accuracy score มากที่สุดสำหรับ dataset นี้

In [ ]:
from datetime import datetime, timedelta
from collections import defaultdict

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import PassiveAggressiveClassifier, RidgeClassifierCV, SGDClassifier, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

# hacking for dataset that already tokenized ;)
def tokenized_tokenizer(text):
  return text

tfidf_params = dict( 
                    min_df=1,
                    max_df=0.8,
                    max_features=None,
                    norm='l2', 
                    ngram_range=(1,3), 
                    lowercase=False,
                    tokenizer=tokenized_tokenizer,
                    use_idf=True,
                    smooth_idf=True,
                    sublinear_tf=True,
                    )

models = [
  # PassiveAggressiveClassifier(), 
  # RidgeClassifierCV(),
  # LogisticRegressionCV(),
  # SGDClassifier(),
  # MultinomialNB(),
  SVC(C=1, gamma='scale', kernel='rbf', degree=2, max_iter=-1),
  RandomForestClassifier(n_estimators=200),
  # BaggingClassifier(),
  # KNeighborsClassifier(),
]

benchmark = defaultdict(list)

for model in models:

    pipe = Pipeline(steps=[
        ('features', TfidfVectorizer(**tfidf_params)),
        ('model', model),
    ])

    train_start_time = datetime.now()

    pipe.fit(X_train, y_train)
    score = pipe.score(X_test, y_test)

    train_end_time = datetime.now()
    train_duration = train_end_time - train_start_time

    model_name = pipe.named_steps['model'].__class__.__name__
    model_params = pipe.named_steps['model']. get_params()

    print(f'{model_name} Scored: {score:.3f} Time: {train_duration.seconds}s\n')

    benchmark['model_name'].append(model_name)
    benchmark['score'].append(score)
    benchmark['time(s)'].append(train_duration.seconds)
    benchmark['model'].append(pipe)
    benchmark['used_params'].append(model_params)


models_df = pd.DataFrame(benchmark)
models_df.sort_values(by=['score', 'time(s)'], ascending=False)

SVC Scored: 0.810 Time: 347s

RandomForestClassifier Scored: 0.789 Time: 723s



,model_name,score,time(s),model,used_params
0,SVC,0.809524,347,"(TfidfVectorizer(lowercase=False, max_df=0.8, ...","{'C': 1, 'break_ties': False, 'cache_size': 20..."
1,RandomForestClassifier,0.789216,723,"(TfidfVectorizer(lowercase=False, max_df=0.8, ...","{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."


# ตัวอย่างการใช้งาน

In [ ]:
from pythainlp.tokenize import word_tokenize
import numpy as np

# ตัวอย่างประโยค fake news ผมคิดมั่วเองแต่ก็อาศัย pattern ที่เห็นแชร์ในกลุ่ม Line บ่อย ๆ นั่นแหละ :p
sentences = [
  'อึ้ง!!! แชร์เก็บไว้เลย!!! ข่าวจากกระทรวงสาธารณสุขประเทศวานูอาตู มะนาวรักษามะเร็งได้จริง รีบแชร์ให้ครอบครัว คนที่คุณรัก',
  'กรมควบคุมโรคเตือน หน้าร้อนนี้อากาศร้อนจัด ให้เฝ้าระวังอาการเจ็บป่วยจากฮีทสโตรก แนะนำอย่าตากแดดเป็นเวลานาน หมั่นดื่มน้ำทั้งวันอย่าให้ร่างกายขาดน้ำ หลีกเลี่ยงการอยู่ในที่ที่อุณหภูมิเปลี่ยนแปลงเร็วเกินไป',
  'ความผิดพลาดของระบบนี้เกิดจากวงโคจรของดาวเทียมเกิดอยู่ในตำแหน่งที่ตรวจพบการสะท้อนของแสงอาทิตย์บนเมฆที่อยู่ระดับสูงเหนือมลรัฐนอร์ทดาโคตาคล้ายคลึงกับเส้นแนวไอจรวดของขีปนาวุธ ระบบจึงได้เข้าใจผิดว่ามันคือขีปนาวุธที่วิ่งเข้ามา',
]

# ทำไมใช้ engine ตัวตัดคำเป็น ICU? เพราะ dataset ต้นทางตัดคำด้วย ICU จึงจำเป็นต้องใช้ ICU เพื่อให้ตัดคำได้เหมือนกัน
tokenized_sentences = [word_tokenize(sent, engine='icu') for sent in sentences]

highest_score_model = models_df.iloc[0].model
predictions = highest_score_model.predict(tokenized_sentences)

predictions

array(['Fake News', 'Fact News', 'Undefined'], dtype=object)

# Hyperparameter Tuning

เนื่องจากรัน GridSearchCV ใช้เวลานานมาก ยิ่งจำนวน parameter มาก combination ก็ยิ่งมากตามไปด้วย ถ้าไม่อยากรันสามารถใช้ตามข้างล่างนี้ได้เลย

```
Best score: 0.813
Best parameters set:
	clf__C: 1
	clf__gamma: 'scale'
	clf__kernel: 'rbf'
	tfidf__lowercase: False
	tfidf__max_df: 0.8
	tfidf__max_features: None
	tfidf__min_df: 10
	tfidf__ngram_range: (1, 3)
	tfidf__norm: 'l2'
	tfidf__smooth_idf: True
	tfidf__sublinear_tf: True
	tfidf__tokenizer: <function tokenized_tokenizer at 0x7fb6d27f2c20>
	tfidf__use_idf: True
```

In [ ]:
# # Parameters Tuning

# from sklearn.model_selection import GridSearchCV

# gs_pipe = Pipeline([
#   ("tfidf", TfidfVectorizer()),
#   ("clf", SVC()),
# ])

# gs_params = {
#   'tfidf__max_df': [.8],
#   'tfidf__min_df': [10],
#   'tfidf__max_features': [None],
#   'tfidf__use_idf': [True],
#   'tfidf__norm': ['l2'],
#   'tfidf__smooth_idf': [True],
#   'tfidf__sublinear_tf': [True],
#   'tfidf__ngram_range': [(1,2), (1,3)],
#   'tfidf__tokenizer': [tokenized_tokenizer],
#   'tfidf__lowercase': [False],
#   'clf__C': [1],
#   'clf__kernel': ['rbf'],
#   # 'clf__max_iter': [-1, 100, 1000],
#   'clf__gamma': ['scale'],
# }

# gs = GridSearchCV(gs_pipe, gs_params)

# gs.fit(X_train, y_train)

# print("Best score: %0.3f" % gs.best_score_)
# print("Best parameters set:")

# best_params = gs.best_estimator_.get_params()

# for param_name in sorted(gs_params.keys()):
#   print("\t%s: %r" % (param_name, best_params[param_name]))

Best score: 0.813
Best parameters set:
	clf__C: 1
	clf__gamma: 'scale'
	clf__kernel: 'rbf'
	tfidf__lowercase: False
	tfidf__max_df: 0.8
	tfidf__max_features: None
	tfidf__min_df: 10
	tfidf__ngram_range: (1, 3)
	tfidf__norm: 'l2'
	tfidf__smooth_idf: True
	tfidf__sublinear_tf: True
	tfidf__tokenizer: <function tokenized_tokenizer at 0x7fb6d27f2c20>
	tfidf__use_idf: True
